In [1]:
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from datetime import datetime

from lmfit import Minimizer, Parameters, report_fit
import chart_studio.plotly as py

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import cufflinks as cf

In [2]:
def Cauchy_cumulative_hazard_fit(x,loc,scale,decaybase):
    decayterm=np.power(decaybase,(x-loc))
    decayterm[np.where((x-loc)<0)]=1.0
    z=(x-loc)/(scale*decayterm)
    CHF=-np.log(0.5 - np.arctan(z)/np.pi)
    return CHF

# define objective function: returns the array to be minimized
def Cauchy_cumulative_hazard_residual(params, x, data):
    """Model a decaying sine wave and subtract data."""
    amp = params['amp']
    loc = params['loc']
    scale = params['scale']
    decaybase=params['decaybase']
    decayterm=np.power(decaybase,(x-loc))
    decayterm[np.where((x-loc)<0)]=1.0
    z=(x-loc)/(scale*decayterm)
    CHF=-np.log(0.5 - np.arctan(z)/np.pi)
    model = amp * CHF
    return model - data

In [3]:
# data generated by run3
datafile='/media/Data1/pyWorkDir/Bigdata/Pyspark/data/COVID19/Canada_Province_Cases.csv'
data0=pd.read_csv(datafile)
data0=data0.drop('Repatriated',axis=1)
data0.columns


Index(['Date', 'Alberta', 'BC', 'Manitoba', 'NL', 'NWT', 'New Brunswick',
       'Nova Scotia', 'Ontario', 'PEI', 'Quebec', 'Saskatchewan', 'Yukon',
       'CA'],
      dtype='object')

In [4]:
data0['Date']=pd.to_datetime(data0['Date'])
data0=data0.set_index('Date')
data0.head()

,Alberta,BC,Manitoba,NL,NWT,New Brunswick,Nova Scotia,Ontario,PEI,Quebec,Saskatchewan,Yukon,CA
Date,,,,,,,,,,,,,
2020-03-01,0,8,0,0,0,0,0,15,0,1,0,0,24
2020-03-03,0,12,0,0,0,0,0,20,0,1,0,0,33
2020-03-04,0,13,0,0,0,0,0,20,0,1,0,0,34
2020-03-05,1,21,0,0,0,0,0,22,0,2,0,0,46
2020-03-06,2,21,0,0,0,0,0,28,0,3,0,0,54


In [5]:
TheProv="Saskatchewan"
data=data0[data0[TheProv]>1]

titletxt="Projected COVID-19 Cases in " + TheProv
filenametxt="ProjectedCOVID19_" + TheProv
start_date=data.index[0].date()
end_date=data.index[-1].date()


data=data[TheProv]
dateData=pd.date_range(start=start_date,end=end_date)
forecastDays=60
dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
dateObsForecast=dateData.append(dateForecast)
last=data.iloc[-1]
data=data/last
dataLen=data.count()
x = np.linspace(1, dataLen, dataLen)

# create a set of Parameters
params = Parameters()
params.add('amp', value=1)
params.add('scale', value=1, min=0.1)
params.add('loc', value=70, min=0)
params.add('decaybase',value=1.0,min=0.95,max=1.0075)  # shparly increase value>1

# do fit, here with the default leastsq algorithm
minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
result = minner.minimize()

# calculate final result
forecastdays=forecastDays
final = data + result.residual
x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
scale=result.params['scale'].value
amp=result.params['amp'].value
loc=result.params['loc'].value
decaybase=result.params['decaybase'].value

y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
nn=np.int(np.ceil(loc))
y1[:nn]=final.iloc[:nn] # replace data before
# topdate=y1[1:]-y1[:-1]
# topdate[topdate<0]
report_fit(result)
amp*last

output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
output

output['date']=dateObsForecast
output['Forecast']=y1*last
output['Cases'].iloc[:dataLen]=data.values*last
output['Fitting'].iloc[:dataLen]=final.values*last
output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
output=output.set_index('date')

output.iplot(asFigure=True,
             mode='lines+markers',
             size=3,secondary_y = 'Increase',
             secondary_y_title='Increase',
             xTitle='Date',
             yTitle='Cases',
             title=titletxt)

@interact
def plot_ProjectedSaskatchewanCOVID19():
    fig=output.iplot(asFigure=True,
                 mode='lines+markers',
                 size=3,secondary_y = 'Increase',
                 secondary_y_title='Increase',
                 xTitle='Date',
                 yTitle='Cases',
                 title=titletxt,
                 theme='solar',
                 filename=filenametxt,
                 sharing='public')
    fig.show()
    url=py.iplot(fig, filename=filenametxt)
    print(url)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 244
    # data points      = 56
    # variables        = 4
    chi-square         = 0.10241747
    reduced chi-square = 0.00196957
    Akaike info crit   = -345.026779
    Bayesian info crit = -336.925373
[[Variables]]
    amp:        0.14432209 +/- 0.03137293 (21.74%) (init = 1)
    scale:      4.23736296 +/- 1.18051039 (27.86%) (init = 1)
    loc:        10.5525255 +/- 1.10563291 (10.48%) (init = 70)
    decaybase:  0.95000000 +/- 0.00841064 (0.89%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, decaybase)   =  0.982
    C(loc, decaybase)   =  0.804
    C(amp, scale)       =  0.792
    C(amp, loc)         =  0.760
    C(scale, decaybase) =  0.678
    C(scale, loc)       =  0.298


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [6]:
TheProv="Nova Scotia"
data=data0[data0[TheProv]>1]

titletxt="Projected COVID-19 Cases in " + TheProv
filenametxt="ProjectedCOVID19_" + TheProv
start_date=data.index[0].date()
end_date=data.index[-1].date()


data=data[TheProv]
dateData=pd.date_range(start=start_date,end=end_date)
forecastDays=60
dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
dateObsForecast=dateData.append(dateForecast)
last=data.iloc[-1]
data=data/last
dataLen=data.count()
x = np.linspace(1, dataLen, dataLen)

# create a set of Parameters
params = Parameters()
params.add('amp', value=1)
params.add('scale', value=1, min=0.1)
params.add('loc', value=70, min=0)
params.add('decaybase',value=1.0,min=0.85,max=1.0075)  # shparly increase value>1

# do fit, here with the default leastsq algorithm
minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
result = minner.minimize()

# calculate final result
forecastdays=forecastDays
final = data + result.residual
x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
scale=result.params['scale'].value
amp=result.params['amp'].value
loc=result.params['loc'].value
decaybase=result.params['decaybase'].value

y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
nn=np.int(np.ceil(loc))
y1[:nn]=final.iloc[:nn] # replace data before
# topdate=y1[1:]-y1[:-1]
# topdate[topdate<0]
report_fit(result)
amp*last

output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
output

output['date']=dateObsForecast
output['Forecast']=y1*last
output['Cases'].iloc[:dataLen]=data.values*last
output['Fitting'].iloc[:dataLen]=final.values*last
output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
output=output.set_index('date')

output.iplot(asFigure=True,
             mode='lines+markers',
             size=3,secondary_y = 'Increase',
             secondary_y_title='Increase',
             xTitle='Date',
             yTitle='Cases',
             title=titletxt)

@interact
def plot_ProjectedNovaScotiaCOVID19():
    fig=output.iplot(asFigure=True,
                 mode='lines+markers',
                 size=3,secondary_y = 'Increase',
                 secondary_y_title='Increase',
                 xTitle='Date',
                 yTitle='Cases',
                 title=titletxt,
                 theme='solar',
                 filename=filenametxt,
                 sharing='public')
    fig.show()
    url=py.iplot(fig, filename=filenametxt)
    print(url)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 140
    # data points      = 54
    # variables        = 4
    chi-square         = 0.03246930
    reduced chi-square = 6.4939e-04
    Akaike info crit   = -392.487995
    Bayesian info crit = -384.532059
[[Variables]]
    amp:        0.31247876 +/- 0.04271634 (13.67%) (init = 1)
    scale:      7.66665751 +/- 0.45114992 (5.88%) (init = 1)
    loc:        21.1344722 +/- 1.23933376 (5.86%) (init = 70)
    decaybase:  0.97790024 +/- 0.01194599 (1.22%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, decaybase)   =  0.986
    C(amp, loc)         =  0.976
    C(loc, decaybase)   =  0.973
    C(amp, scale)       =  0.398
    C(scale, loc)       =  0.266
    C(scale, decaybase) =  0.257


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [7]:
TheProv="Quebec"
data=data0[data0[TheProv]>1]

titletxt="Projected COVID-19 Cases in " + TheProv
filenametxt="ProjectedCOVID19_" + TheProv
start_date=data.index[0].date()
end_date=data.index[-1].date()


data=data[TheProv]
dateData=pd.date_range(start=start_date,end=end_date)
forecastDays=60
dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
dateObsForecast=dateData.append(dateForecast)
last=data.iloc[-1]
data=data/last
dataLen=data.count()
x = np.linspace(1, dataLen, dataLen)

# create a set of Parameters
params = Parameters()
params.add('amp', value=1)
params.add('scale', value=1, min=0.1)
params.add('loc', value=70, min=0)
params.add('decaybase',value=1.0,min=0.85,max=1.0075)  # shparly increase value>1

# do fit, here with the default leastsq algorithm
minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
result = minner.minimize()

# calculate final result
forecastdays=forecastDays
final = data + result.residual
x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
scale=result.params['scale'].value
amp=result.params['amp'].value
loc=result.params['loc'].value
decaybase=result.params['decaybase'].value

y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
nn=np.int(np.ceil(loc))
y1[:nn]=final.iloc[:nn] # replace data before
# topdate=y1[1:]-y1[:-1]
# topdate[topdate<0]
report_fit(result)
amp*last

output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
output

output['date']=dateObsForecast
output['Forecast']=y1*last
output['Cases'].iloc[:dataLen]=data.values*last
output['Fitting'].iloc[:dataLen]=final.values*last
output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
output=output.set_index('date')

output.iplot(asFigure=True,
             mode='lines+markers',
             size=3,secondary_y = 'Increase',
             secondary_y_title='Increase',
             xTitle='Date',
             yTitle='Cases',
             title=titletxt)

@interact
def plot_ProjectedQuebecCOVID19():
    fig=output.iplot(asFigure=True,
                 mode='lines+markers',
                 size=3,secondary_y = 'Increase',
                 secondary_y_title='Increase',
                 xTitle='Date',
                 yTitle='Cases',
                 title=titletxt,
                 theme='solar',
                 filename=filenametxt,
                 sharing='public')
    fig.show()
    url=py.iplot(fig, filename=filenametxt)
    print(url)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 212
    # data points      = 64
    # variables        = 4
    chi-square         = 0.04629290
    reduced chi-square = 7.7155e-04
    Akaike info crit   = -454.825591
    Bayesian info crit = -446.190059
[[Variables]]
    amp:        0.10760290 +/- 0.00409205 (3.80%) (init = 1)
    scale:      9.59712966 +/- 1.77266053 (18.47%) (init = 1)
    loc:        25.9472928 +/- 0.64501613 (2.49%) (init = 70)
    decaybase:  0.85000000 +/- 0.00574756 (0.68%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, decaybase)   =  0.851
    C(scale, loc)       = -0.773
    C(loc, decaybase)   =  0.300
    C(amp, scale)       =  0.242
    C(amp, loc)         =  0.210
    C(scale, decaybase) = -0.110


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [8]:
TheProv="Manitoba"
data=data0[data0[TheProv]>1]

titletxt="Projected COVID-19 Cases in " + TheProv
filenametxt="ProjectedCOVID19_" + TheProv
start_date=data.index[0].date()
end_date=data.index[-1].date()


data=data[TheProv]
dateData=pd.date_range(start=start_date,end=end_date)
forecastDays=60
dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
dateObsForecast=dateData.append(dateForecast)
last=data.iloc[-1]
data=data/last
dataLen=data.count()
x = np.linspace(1, dataLen, dataLen)

# create a set of Parameters
params = Parameters()
params.add('amp', value=1)
params.add('scale', value=1, min=0.1)
params.add('loc', value=70, min=0)
params.add('decaybase',value=1.0,min=0.85,max=1.0075)  # shparly increase value>1

# do fit, here with the default leastsq algorithm
minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
result = minner.minimize()

# calculate final result
forecastdays=forecastDays
final = data + result.residual
x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
scale=result.params['scale'].value
amp=result.params['amp'].value
loc=result.params['loc'].value
decaybase=result.params['decaybase'].value

y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
nn=np.int(np.ceil(loc))
y1[:nn]=final.iloc[:nn] # replace data before
# topdate=y1[1:]-y1[:-1]
# topdate[topdate<0]
report_fit(result)
amp*last

output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
output

output['date']=dateObsForecast
output['Forecast']=y1*last
output['Cases'].iloc[:dataLen]=data.values*last
output['Fitting'].iloc[:dataLen]=final.values*last
output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
output=output.set_index('date')

output.iplot(asFigure=True,
             mode='lines+markers',
             size=3,secondary_y = 'Increase',
             secondary_y_title='Increase',
             xTitle='Date',
             yTitle='Cases',
             title=titletxt)

@interact
def plot_ProjectedManitobaCOVID19():
    fig=output.iplot(asFigure=True,
                 mode='lines+markers',
                 size=3,secondary_y = 'Increase',
                 secondary_y_title='Increase',
                 xTitle='Date',
                 yTitle='Cases',
                 title=titletxt,
                 theme='solar',
                 filename=filenametxt,
                 sharing='public')
    fig.show()
    url=py.iplot(fig, filename=filenametxt)
    print(url)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 354
    # data points      = 56
    # variables        = 4
    chi-square         = 0.05654820
    reduced chi-square = 0.00108747
    Akaike info crit   = -378.288765
    Bayesian info crit = -370.187359
[[Variables]]
    amp:        0.27622626 +/- 0.01967124 (7.12%) (init = 1)
    scale:      2.26871045 +/- 0.32634676 (14.38%) (init = 1)
    loc:        15.7216205 +/- 0.25116375 (1.60%) (init = 70)
    decaybase:  1.00750000 +/- 0.00197668 (0.20%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, scale)       =  0.922
    C(amp, decaybase)   = -0.882
    C(loc, decaybase)   = -0.666
    C(scale, decaybase) = -0.646
    C(amp, loc)         =  0.544
    C(scale, loc)       =  0.303


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [9]:
TheProv="BC"
data=data0[data0[TheProv]>11]

titletxt="Projected COVID-19 Cases in " + TheProv
filenametxt="ProjectedCOVID19_" + TheProv
start_date=data.index[0].date()
end_date=data.index[-1].date()


data=data[TheProv]
dateData=pd.date_range(start=start_date,end=end_date)
forecastDays=60
dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
dateObsForecast=dateData.append(dateForecast)
last=data.iloc[-1]
data=data/last
dataLen=data.count()
x = np.linspace(1, dataLen, dataLen)

# create a set of Parameters
params = Parameters()
params.add('amp', value=1)
params.add('scale', value=1, min=0.1)
params.add('loc', value=70, min=0)
params.add('decaybase',value=1.0,min=0.85,max=1.0075)  # shparly increase value>1

# do fit, here with the default leastsq algorithm
minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
result = minner.minimize()

# calculate final result
forecastdays=forecastDays
final = data + result.residual
x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
scale=result.params['scale'].value
amp=result.params['amp'].value
loc=result.params['loc'].value
decaybase=result.params['decaybase'].value

y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
nn=np.int(np.ceil(loc))
y1[:nn]=final.iloc[:nn] # replace data before
# topdate=y1[1:]-y1[:-1]
# topdate[topdate<0]
report_fit(result)
amp*last

output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
output

output['date']=dateObsForecast
output['Forecast']=y1*last
output['Cases'].iloc[:dataLen]=data.values*last
output['Fitting'].iloc[:dataLen]=final.values*last
output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
output=output.set_index('date')

output.iplot(asFigure=True,
             mode='lines+markers',
             size=3,secondary_y = 'Increase',
             secondary_y_title='Increase',
             xTitle='Date',
             yTitle='Cases',
             title=titletxt)

@interact
def plot_ProjectedBCCOVID19():
    fig=output.iplot(asFigure=True,
                 mode='lines+markers',
                 size=3,secondary_y = 'Increase',
                 secondary_y_title='Increase',
                 xTitle='Date',
                 yTitle='Cases',
                 title=titletxt,
                 theme='solar',
                 filename=filenametxt,
                 sharing='public')
    fig.show()
    url=py.iplot(fig, filename=filenametxt)
    print(url)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 766
    # data points      = 66
    # variables        = 4
    chi-square         = 0.00996758
    reduced chi-square = 1.6077e-04
    Akaike info crit   = -572.672749
    Bayesian info crit = -563.914130
[[Variables]]
    amp:        0.11845859 +/- 0.00700661 (5.91%) (init = 1)
    scale:      3.76650154 +/- 0.33522673 (8.90%) (init = 1)
    loc:        15.7568727 +/- 0.27620855 (1.75%) (init = 70)
    decaybase:  0.91002243 +/- 0.00796831 (0.88%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, decaybase)   =  0.989
    C(amp, scale)       =  0.742
    C(loc, decaybase)   =  0.662
    C(scale, decaybase) =  0.652
    C(amp, loc)         =  0.615


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [10]:
output.shape

(126, 4)

In [11]:
TheProv="Alberta"
data=data0[data0[TheProv]>1]
titletxt="Projected COVID-19 Cases in " + TheProv
filenametxt="ProjectedCOVID19_" + TheProv
start_date=data.index[0].date()
end_date=data.index[-1].date()


data=data[TheProv]
dateData=pd.date_range(start=start_date,end=end_date)
forecastDays=60
dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
dateObsForecast=dateData.append(dateForecast)
last=data.iloc[-1]
data=data/last
dataLen=data.count()
x = np.linspace(1, dataLen, dataLen)

# create a set of Parameters
params = Parameters()
params.add('amp', value=1)
params.add('scale', value=1, min=0.1)
params.add('loc', value=70, min=0)
params.add('decaybase',value=1.0,min=0.85,max=1.0075)  # shparly increase value>1

# do fit, here with the default leastsq algorithm
minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
result = minner.minimize()

# calculate final result
forecastdays=forecastDays
final = data + result.residual
x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
scale=result.params['scale'].value
amp=result.params['amp'].value
loc=result.params['loc'].value
decaybase=result.params['decaybase'].value

y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
nn=np.int(np.ceil(loc))
y1[:nn]=final.iloc[:nn] # replace data before
# topdate=y1[1:]-y1[:-1]
# topdate[topdate<0]
report_fit(result)
amp*last

output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
output

output['date']=dateObsForecast
output['Forecast']=y1*last
output['Cases'].iloc[:dataLen]=data.values*last
output['Fitting'].iloc[:dataLen]=final.values*last
output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
output=output.set_index('date')

output.iplot(asFigure=True,
             mode='lines+markers',
             size=3,secondary_y = 'Increase',
             secondary_y_title='Increase',
             xTitle='Date',
             yTitle='Cases',
             title=titletxt)

@interact
def plot_ProjectedAlbertaCOVID19():
    fig=output.iplot(asFigure=True,
                 mode='lines+markers',
                 size=3,secondary_y = 'Increase',
                 secondary_y_title='Increase',
                 xTitle='Date',
                 yTitle='Cases',
                 title=titletxt,
                 theme='solar',
                 filename=filenametxt,
                 sharing='public')
    fig.show()
    url=py.iplot(fig, filename=filenametxt)
    print(url)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 142
    # data points      = 63
    # variables        = 4
    chi-square         = 0.05819962
    reduced chi-square = 9.8643e-04
    Akaike info crit   = -432.181701
    Bayesian info crit = -423.609162
[[Variables]]
    amp:        0.33025662 +/- 0.12860788 (38.94%) (init = 1)
    scale:      12.4717761 +/- 2.80748271 (22.51%) (init = 1)
    loc:        35.8026556 +/- 4.52880178 (12.65%) (init = 70)
    decaybase:  0.95468585 +/- 0.03879623 (4.06%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, decaybase)   =  0.997
    C(amp, loc)         =  0.997
    C(loc, decaybase)   =  0.994
    C(scale, decaybase) = -0.969
    C(scale, loc)       = -0.963
    C(amp, scale)       = -0.954


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [12]:
# TheProv="Ontario"
# data=data0[data0[TheProv]>1]
# titletxt="Projected COVID-19 Cases in " + TheProv
# filenametxt="ProjectedCOVID19_" + TheProv
# start_date=data.index[0].date()
# end_date=data.index[-1].date()


# data=data[TheProv]
# dateData=pd.date_range(start=start_date,end=end_date)
# forecastDays=60
# dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
# dateObsForecast=dateData.append(dateForecast)
# last=data.iloc[-1]
# data=data/last
# dataLen=data.count()
# x = np.linspace(1, dataLen, dataLen)

# # create a set of Parameters
# params = Parameters()
# params.add('amp', value=1)
# params.add('scale', value=1, min=0.1)
# params.add('loc', value=70, min=0)
# params.add('decaybase',value=1.0,min=0.85,max=1.0075)  # shparly increase value>1

# # do fit, here with the default leastsq algorithm
# minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
# result = minner.minimize()

# # calculate final result
# forecastdays=forecastDays
# final = data + result.residual
# x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
# scale=result.params['scale'].value
# amp=result.params['amp'].value
# loc=result.params['loc'].value
# decaybase=result.params['decaybase'].value

# y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
# nn=np.int(np.ceil(loc))
# y1[:nn]=final.iloc[:nn] # replace data before
# # topdate=y1[1:]-y1[:-1]
# # topdate[topdate<0]
# report_fit(result)
# amp*last

# output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
# output

# output['date']=dateObsForecast
# output['Forecast']=y1*last
# output['Cases'].iloc[:dataLen]=data.values*last
# output['Fitting'].iloc[:dataLen]=final.values*last
# output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
# output=output.set_index('date')

# output.iplot(asFigure=True,
#              mode='lines+markers',
#              size=3,secondary_y = 'Increase',
#              secondary_y_title='Increase',
#              xTitle='Date',
#              yTitle='Cases',
#              title=titletxt)

# @interact
# def plot_ProjectedOntarioCOVID19():
#     fig=output.iplot(asFigure=True,
#                  mode='lines+markers',
#                  size=3,secondary_y = 'Increase',
#                  secondary_y_title='Increase',
#                  xTitle='Date',
#                  yTitle='Cases',
#                  title=titletxt,
#                  theme='solar',
#                  filename=filenametxt,
#                  sharing='public')
#     fig.show()
#     url=py.iplot(fig, filename=filenametxt)
#     print(url)